In [1]:
import sys
import os
import pandas as pd 
import numpy as np
from tqdm.notebook import tqdm 

sys.path.insert(0, '/csc/epitkane/projects/multimodal/src/')
from utils import list_files_in_dir

In [3]:
paths = []
paths = list_files_in_dir('/csc/epitkane/projects/multimodal/data/train_new/muat_orig_epipos', paths)
#paths = list_files_in_dir('/csc/epitkane/projects/multimodal/train/motif3', paths)
paths.sort()

In [4]:
counts = [i for i in paths if i.split('/')[-1][:5] == 'count' ]

In [5]:
counts[0]

'/csc/epitkane/projects/multimodal/data/train_new/muat_orig_epipos/Bone-Osteosarc/f221c897-6ad0-0df9-e040-11ac0c4813ef/count_f221c897-6ad0-0df9-e040-11ac0c4813ef.tsv.gz'

In [15]:
SNV = 0
indel = 0 
MNV = 0
for path in tqdm(counts):
    data = pd.read_csv(path, compression='gzip', sep = '\t', index_col=0)
    SNV += data.iloc[0,0]
    indel += data.iloc[0,2]
    MNV += data.iloc[0,1]
    if data.loc[0,'SNV'] ==0 and data.loc[0,'indel']==0 and data.loc[0,'MNV']==0:
        print(data)
        print(path)
        SNV_file = pd.read_csv(os.path.join('/'.join(path.split('/')[:-1]), 'SNV_'+ path.split('/')[-1].split('_')[1]), compression='gzip', sep='\t', low_memory=False)
        indel_file = pd.read_csv(os.path.join('/'.join(path.split('/')[:-1]), 'indel_'+ path.split('/')[-1].split('_')[1]), compression='gzip', sep='\t', low_memory=False)
        MNV_file = pd.read_csv(os.path.join('/'.join(path.split('/')[:-1]), 'MNV_'+ path.split('/')[-1].split('_')[1]), compression='gzip', sep='\t', low_memory=False)
        data.loc[0,'SNV'] = len(SNV_file)
        data.loc[0,'indel'] = len(indel_file)
        data.loc[0,'MNV'] = len(MNV_file)
        data.to_csv(path, compression='gzip', sep='\t')

  0%|          | 0/2587 [00:00<?, ?it/s]

In [16]:
print(f'SNV:\t{SNV}\nMNV:\t{MNV}\nindel:\t{indel}')

SNV:	41126011
MNV:	806986
indel:	3653253


### amounts for train/DNABERT_motif201

In [7]:
print(f'SNV:\t{SNV}\nMNV:\t{MNV}\nindel:\t{indel}')

SNV:	41089087
MNV:	805638
indel:	3650493


### Count the cases and mutations in the train and validation file 

In [3]:
train_cases = pd.read_csv("/csc/epitkane/projects/multimodal/extfiles/fullpcawg_train_checkpoint_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3.csv")
valid_cases = pd.read_csv("/csc/epitkane/projects/multimodal/extfiles/fullpcawg_val_checkpoint_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3.csv")

In [8]:
train_1 = train_cases[train_cases.loc[:, 'fold'] == 1]
valid_1 = valid_cases[valid_cases.loc[:, 'fold'] == 1]

In [9]:
data = pd.concat([train_1, valid_1])

In [10]:
data

,Unnamed: 0,samples,nm_class,slices,fold
0,0,f82d213f-9843-28eb-e040-11ac0d483e48.csv,Bone-Osteosarc,2,1
1,1,f854ae3f-356d-f0ad-e040-11ac0c485dae.csv,Bone-Osteosarc,9,1
2,0,f82d213f-caa7-fd59-e040-11ac0d483e46.csv,Bone-Osteosarc,6,1
3,0,f83f1eb8-b2c8-c3d6-e040-11ac0c48261e.csv,Bone-Osteosarc,9,1
4,3,f8809cee-8f81-28f8-e040-11ac0d4873bf.csv,Bone-Osteosarc,8,1
...,...,...,...,...,...
254,254,15d597de-d26c-4ce1-b0cb-077e39b55f30.csv,Uterus-AdenoCA,1,1
255,255,55ada396-0878-4724-8ebe-00da367eed3d.csv,Uterus-AdenoCA,1,1
256,256,8dd14f0e-8601-4aa1-864c-3c49e768cdd1.csv,Uterus-AdenoCA,1,1
257,257,b0a83df8-dd2c-4c1b-b238-9081d2c22258.csv,Uterus-AdenoCA,1,1


In [32]:
count_types = {}
for case in tqdm(data.loc[:,'nm_class' ].values):
    if case not in count_types.keys():
        count_types[case] = [1]
    else:
        count_types[case][0] += 1
    

KeyError: 'nm_class'

In [20]:
df = pd.DataFrame(count_types)
df.T

,0
Bone-Osteosarc,44
Breast-AdenoCA,197
CNS-GBM,41
CNS-Medullo,146
CNS-PiloAstro,89
ColoRect-AdenoCA,58
Eso-AdenoCA,98
Head-SCC,57
Kidney-ChRCC,45
Kidney-RCC,144


In [27]:
samples = [i[:-4] for i in data.loc[:,'samples' ].values]

In [33]:
SNV = 0
indel = 0 
MNV = 0
for path in tqdm(counts):
    if path.split('/')[-2] in samples:
        data = pd.read_csv(path, compression='gzip', sep = '\t', index_col=0)
        SNV += data.iloc[0,0]
        indel += data.iloc[0,2]
        MNV += data.iloc[0,1]

  0%|          | 0/2587 [00:00<?, ?it/s]

In [34]:
print(f'SNV:\t{SNV}\nMNV:\t{MNV}\nindel:\t{indel}')

SNV:	40625000
MNV:	797405
indel:	3604428


## Check the numpy files

In [25]:
paths = []
paths = list_files_in_dir('/csc/epitkane/projects/multimodal/data/train_new/DNABERT_motif7', paths)
paths.sort()

In [26]:
numpy_files = []
tsv_files = []
for i in paths:
    if i.split('/')[-1].split('.')[-1] == 'npz':
        numpy_files.append(i)
    if i.split('/')[-1].split('.')[-1] == 'gz' and i.split('/')[-1][:5] != 'count':
        tsv_files.append(i)

In [27]:
len(numpy_files) == len(tsv_files)

True

In [ ]:
for path in tqdm(numpy_files):
    try:
        array = np.load(path)
        if array['motif'].shape[0] == 0 or array['position'].shape[0] == 0 or array['GES'].shape[0] == 0:
            print(path)
    except Exception as e:
        print(path)
        raise e

In [ ]:
faulty_array_files = []
data_files = []
for np_path, counts in tqdm(zip(numpy_files, tsv_files), total =len(numpy_files)):
    array = np.load(np_path)['motif']
    count = pd.read_csv(counts, compression='gzip', sep='\t', index_col=0, header=0, low_memory=False)
    #mut_type = np_path.split('/')[-1].split('_')[0]
    if len(count) != len(array):
        faulty_array_files.append(np_path)
        data_files.append(counts)
        print(f'Numpy: {len(array)}, Counts: {len(count)}, path: {counts}')
        

  0%|          | 0/7651 [00:00<?, ?it/s]

## Fix count files

In [2]:
paths = []
paths = list_files_in_dir('/csc/epitkane/projects/multimodal/data/train_new/DNABERT_motif3_diff', paths)
paths.sort()

In [3]:
numpy_files = []
count_files = []
for i in paths:        
    if i.split('/')[-1].split('.')[-1] == 'npz':
        numpy_files.append(i)
        count_files.append('/'.join(i.split('/')[:-1]) + '/count_' + i.split('/')[-1].split('_')[-1][:-4]+ '.tsv.gz')

In [4]:
count_files[1000]

'/csc/epitkane/projects/multimodal/data/train_new/DNABERT_motif3_diff/CNS-Medullo/5a5a8409-a8f2-4c7b-85e5-b8bd85f2930c/count_5a5a8409-a8f2-4c7b-85e5-b8bd85f2930c.tsv.gz'

In [5]:
numpy_files[1000]

'/csc/epitkane/projects/multimodal/data/train_new/DNABERT_motif3_diff/CNS-Medullo/5a5a8409-a8f2-4c7b-85e5-b8bd85f2930c/diff_MNV_5a5a8409-a8f2-4c7b-85e5-b8bd85f2930c.npz'

### generate count files

In [11]:
for np_path, count in tqdm(zip(numpy_files, count_files), total=len(numpy_files)):
    array = np.load(np_path)['motif']
    tumour = np_path.split("/")[-3]
    if not os.path.exists(count):
        count_df = pd.DataFrame.from_dict({'SNV':[0], 'MNV':[0], 'indel':[0], 'SV/MEI':[0], 'Normal':[0]})
    else:
        count_df = pd.read_csv(count, compression='gzip', sep = '\t', index_col=0)
    mutation_type= np_path.split("/")[-1].split("_")[1]
    count_df.loc[0,mutation_type] = len(array)
    count_df.to_csv(count, compression='gzip', sep='\t' )

  0%|          | 0/7651 [00:00<?, ?it/s]

### fix count file

In [ ]:
for np_path, counts in tqdm(zip(numpy_files, count_files), total =len(numpy_files)):
    array = np.load(np_path)['motif']
    count = pd.read_csv(counts, compression='gzip', sep='\t', index_col=0, header=0, low_memory=False)
    mut_type = np_path.split('/')[-1].split('_')[0]
    if int(count.loc[0, mut_type]) != len(array):
        print(f'count {int(count.loc[0, mut_type])}, array {len(array)}')
        count.loc[0, mut_type] = len(array)
        print(count)
        count.to_csv(counts, compression='gzip', sep='\t')

In [7]:
tumour_mut_count = {}
for path in tqdm(counts):
    tumour = path.split("/")[-3]
    if tumour not in tumour_mut_count.keys():
        tumour_mut_count[tumour] = {'SNV':0, 'MNV':0, 'indel':0, 'total':0, 'number of samples':0}
    data = pd.read_csv(path, compression='gzip', sep = '\t')
    tumour_mut_count[tumour]['SNV'] += data.iloc[0,1]
    tumour_mut_count[tumour]['indel'] += data.iloc[0,3]
    tumour_mut_count[tumour]['MNV'] += data.iloc[0,2]
    tumour_mut_count[tumour]['total'] += data.iloc[0,1]+ data.iloc[0,3] + data.iloc[0,2]
    tumour_mut_count[tumour]['number of samples'] += 1

  0%|          | 0/2587 [00:00<?, ?it/s]

In [12]:
for tumour in tumour_mut_count.keys():
    tumour_mut_count[tumour]['SNV'] /= tumour_mut_count[tumour]['number of samples']
    tumour_mut_count[tumour]['indel'] /= tumour_mut_count[tumour]['number of samples']
    tumour_mut_count[tumour]['MNV'] /= tumour_mut_count[tumour]['number of samples']
    tumour_mut_count[tumour]['total'] /= tumour_mut_count[tumour]['number of samples']

In [13]:
df = pd.DataFrame(tumour_mut_count)

In [14]:
df.T

,SNV,MNV,indel,total,number of samples
Bone-Osteosarc,3654.295455,39.840909,178.818182,3872.954545,44.0
Breast-AdenoCA,7062.472081,74.944162,443.852792,7581.269036,197.0
CNS-GBM,12358.585366,47.975610,495.414634,12901.975610,41.0
CNS-Medullo,1378.876712,7.315068,160.452055,1546.643836,146.0
CNS-PiloAstro,236.910112,1.269663,10.808989,248.988764,89.0
ColoRect-AdenoCA,91166.155172,231.913793,14941.017241,106339.086207,58.0
Eso-AdenoCA,26750.040816,177.704082,1519.448980,28447.193878,98.0
Head-SCC,15357.087719,226.333333,672.140351,16255.561404,57.0
Kidney-ChRCC,2597.288889,23.977778,171.133333,2792.400000,45.0
Kidney-RCC,6062.034722,163.493056,860.256944,7085.784722,144.0


In [15]:
print(f'Total size of the dataset: {SNV+MNV+indel}')

Total size of the dataset: 45708603


In [3]:
paths101 = []
paths101 = list_files_in_dir('/csc/epitkane/projects/multimodal/data/train/DNABERT_motif101', paths101)
paths101 = [i.split('/')[-1] for i in paths101 if i.split('/')[-1][:3] == 'SNV' ]
paths3 = []
paths3 = list_files_in_dir('/csc/epitkane/projects/multimodal/data/train/DNABERT_motif3', paths3)
paths3 = [i.split('/')[-1] for i in paths3 if i.split('/')[-1][:3] == 'SNV' ]

In [4]:
paths101.sort()
paths3.sort()

In [5]:
print(len(paths101))
print(len(paths3))

5174
5174


In [6]:
paths101 == paths3

True

In [26]:
for i in range(100, 10000):
    if (41126011+806986+3653253)%i ==0:
        print(i)

125
250
625
1250


In [50]:
(41131719 + 806735 + 3651716)/3635

12542.0

## Check epipos data

In [15]:
epipos_orders = []
epipos_orders = list_files_in_dir('/csc/epitkane/projects/multimodal/data/train_new/test_epipos', epipos_orders)
epipos_orders = [i for i in epipos_orders if i[-7:] == '.tsv.gz']

In [16]:
muat_files = []
muat_files = list_files_in_dir('/csc/epitkane/projects/multimodal/data/train_new/muat_orig_epipos', muat_files)
muat_files = [i for i in muat_files if i[-7:] == '.tsv.gz' and i.split('/')[-1].split('_')[0] != 'count' ]

In [17]:
epipos_orders.sort()
muat_files.sort()

In [27]:
for (e, m) in tqdm(zip(epipos_orders, muat_files), total= len(epipos_orders)):
    assert '_'.join([e.split('/')[-1][:-7].split('_')[0],  e.split('/')[-1][:-7].split('_')[-1]]) == m.split('/')[-1][:-7], f'Not the same file'
    epi = pd.read_csv(e, compression='gzip', sep='\t', index_col= 0, low_memory= False)
    muat = pd.read_csv(m, compression='gzip', sep='\t', index_col= 0, low_memory= False)
    epi = epi.astype(str)
    muat = muat.astype(str)
    if not epi.loc[:,['chrom', 'pos']].equals(muat.loc[:,['chrom', 'pos']]):
        print(e)

  0%|          | 0/7649 [00:00<?, ?it/s]

/csc/epitkane/projects/multimodal/data/train_new/test_epipos/ColoRect-AdenoCA/31e63f89-a6a9-40fb-823d-f41587bd73d8/SNV_order_31e63f89-a6a9-40fb-823d-f41587bd73d8.tsv.gz


In [28]:
e = pd.read_csv('/csc/epitkane/projects/multimodal/data/train_new/test_epipos/ColoRect-AdenoCA/31e63f89-a6a9-40fb-823d-f41587bd73d8/SNV_order_31e63f89-a6a9-40fb-823d-f41587bd73d8.tsv.gz', compression='gzip', sep='\t', index_col= 0, low_memory= False)
m = e = pd.read_csv('/csc/epitkane/projects/multimodal/data/train_new/muat_orig_epipos/ColoRect-AdenoCA/31e63f89-a6a9-40fb-823d-f41587bd73d8/SNV_31e63f89-a6a9-40fb-823d-f41587bd73d8.tsv.gz', compression='gzip', sep='\t', index_col= 0, low_memory= False)

In [29]:
e = e.astype(str)
m = m.astype(str)

In [31]:
e.loc[:,['chrom', 'pos']].equals(m.loc[:,['chrom', 'pos']])

True

In [33]:
a = np.loadtxt('/csc/epitkane/projects/multimodal/data/test_epipos_fix/ColoRect-AdenoCA/0980e7fd-051d-45e9-9ca6-2baf073da4e8/SNV_0980e7fd-051d-45e9-9ca6-2baf073da4e8.txt.gz')

In [34]:
a.shape

(823655, 50)

In [2]:
epipos_orders = []
epipos_orders = list_files_in_dir('/csc/epitkane/projects/multimodal/data/train_new/muat_orig_epipos/epipos', epipos_orders)
epipos_orders = [i for i in epipos_orders if i[-7:] == '.txt.gz']

In [3]:
for e in tqdm(epipos_orders):
    h = np.loadtxt(e)
    if len(h.shape) != 2:
        h = h.reshape(1,50)
        print(e)
        print(h.shape)
        np.savetxt(e, h)

  0%|          | 0/7649 [00:00<?, ?it/s]

/csc/epitkane/projects/multimodal/data/train_new/muat_orig_epipos/epipos/CNS-Medullo/13446fc4-9fa6-4bb3-a00a-b9265b41eb12/MNV_13446fc4-9fa6-4bb3-a00a-b9265b41eb12.txt.gz
(1, 50)
/csc/epitkane/projects/multimodal/data/train_new/muat_orig_epipos/epipos/CNS-PiloAstro/64e5ee38-e5c6-4911-ad87-c50dd8e52f1b/indel_64e5ee38-e5c6-4911-ad87-c50dd8e52f1b.txt.gz
(1, 50)
/csc/epitkane/projects/multimodal/data/train_new/muat_orig_epipos/epipos/CNS-PiloAstro/718c5a31-6b19-4f68-9fb9-79b0c00d64a5/indel_718c5a31-6b19-4f68-9fb9-79b0c00d64a5.txt.gz
(1, 50)
/csc/epitkane/projects/multimodal/data/train_new/muat_orig_epipos/epipos/CNS-PiloAstro/d4ecbf52-f6a9-4394-9d3b-8eb8eee1b17b/indel_d4ecbf52-f6a9-4394-9d3b-8eb8eee1b17b.txt.gz
(1, 50)
/csc/epitkane/projects/multimodal/data/train_new/muat_orig_epipos/epipos/CNS-PiloAstro/d884d070-6077-4155-9978-76aca7ba1e76/indel_d884d070-6077-4155-9978-76aca7ba1e76.txt.gz
(1, 50)
/csc/epitkane/projects/multimodal/data/train_new/muat_orig_epipos/epipos/CNS-PiloAstro/f54d6b6

## Testing groundtruth model

In [2]:
import os

In [3]:
def count_accuracy(df, targets):
    correct = 0
    logits = df.loc[:,'Bone-Osteosarc':'Uterus-AdenoCA']
    for i in range(0, len(df)):
        target = targets.iloc[i]
        row_logits = logits.loc[i,:]
        #print(row_logits)
        pred = row_logits.idxmax()
        #print(pred)
        #print(target)
        if target == pred:
            correct += 1
    return correct/len(df)

In [67]:
acc = 0
for i in range(1,11):
    base = f'/csc/epitkane/projects/litegpt/ckptpuhti/fullpcawgfold{i}_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3/'
    ground_truth = pd.read_csv(os.path.join(base, "probability.csv"), index_col=0)
    targets = pd.read_csv(os.path.join(base, "target.csv"))
    files = pd.read_csv(os.path.join(base, "listsample.csv")) 
    files = pd.DataFrame({'samples':files.loc[:,'0'].values.tolist()})
    info = pd.read_csv("/csc/epitkane/projects/multimodal/data/utils/classinfo_pcawg_.csv")
    names = targets.merge(info, left_on='0', right_on='class_index', how='left')
    names = names.loc[:,'class_name'].values.tolist()
    files['nm_class'] =names
    files['slices'] = [1]*len(files)
    files['fold'] = [1]*len(files)
    acc += count_accuracy(ground_truth, files.loc[:,'nm_class'])
print(acc/10)

0.8844183053485379


In [68]:
base = '/csc/epitkane/projects/litegpt/ckptpuhti/fullpcawgfold9_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3/'

In [69]:
ground_truth = pd.read_csv(os.path.join(base, "probability.csv"), index_col=0)

In [70]:
targets = pd.read_csv(os.path.join(base, "target.csv"))
files = pd.read_csv(os.path.join(base, "listsample.csv")) 
files = pd.DataFrame({'samples':files.loc[:,'0'].values.tolist()})
info = pd.read_csv("/csc/epitkane/projects/multimodal/data/utils/classinfo_pcawg_.csv")
names = targets.merge(info, left_on='0', right_on='class_index', how='left')
names = names.loc[:,'class_name'].values.tolist()
files['nm_class'] =names
files['slices'] = [1]*len(files)
files['fold'] = [1]*len(files)

In [71]:
count_accuracy(ground_truth, files.loc[:,'nm_class'])

0.9069767441860465

In [9]:
#files

In [10]:
#files.to_csv("/csc/epitkane/projects/multimodal/extfiles/pcawg_val_test.csv")

In [64]:
PCAWG = pd.read_csv("/csc/epitkane/projects/multimodal/extfiles/fullpcawg_val_checkpoint_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3.csv", index_col=0)

In [65]:
df = pd.DataFrame({'model':PCAWG[PCAWG.loc[:,'fold'] ==1].loc[:,'samples'], 'truth':files.loc[:,'samples']})

In [ ]:
#pd.set_option('display.max_rows', None)
df

In [77]:
predicted = pd.read_csv("/csc/epitkane/projects/multimodal/models/MuAtMotifPos_orig_10_fold_fixed_zero_grad/fold_9/val_logits_fold9_best_vallogits.tsv.gz", compression='gzip', sep='\t')
#predicted = pd.read_csv("/csc/epitkane/projects/multimodal/test_model/orig_implementation/val_logits.tsv", sep='\t')
#predicted = pd.read_csv("/csc/epitkane/projects/multimodal/models/MuAtMotifPos_orig_10_fold_fixed_zero_grad/fold_1/val_logits_fold1_best_vallogits.tsv.gz", compression='gzip', sep='\t')

In [78]:
ground_truth

,Bone-Osteosarc,Breast-AdenoCA,CNS-GBM,CNS-Medullo,CNS-PiloAstro,ColoRect-AdenoCA,Eso-AdenoCA,Head-SCC,Kidney-ChRCC,Kidney-RCC,...,Lymph-CLL,Myeloid-MPN,Ovary-AdenoCA,Panc-AdenoCA,Panc-Endocrine,Prost-AdenoCA,Skin-Melanoma,Stomach-AdenoCA,Thy-AdenoCA,Uterus-AdenoCA
0,4.702490,-1.264239,1.609987,0.895722,-1.536368,-6.890939,0.209029,-1.102477,-5.673927,-5.333275,...,1.959397,0.927518,3.141655,8.889136,3.208280,0.789097,3.658869,-0.162928,-3.220208,-7.374795
1,10.900906,2.584677,-1.896338,0.378731,-1.763025,-5.708251,4.254140,1.578215,0.636892,0.465058,...,-1.739323,-0.100079,-5.016763,1.140113,3.486802,-1.617320,5.810186,5.221578,-2.166786,-3.634579
2,10.175936,1.915897,0.069650,-1.110237,-3.590124,-6.150170,3.967411,1.407679,-2.350145,-1.173864,...,-0.232013,-0.749353,-1.310855,2.165340,6.187298,-7.029920,5.779738,3.449163,-1.129770,-2.492158
3,1.401752,-4.835634,-2.212931,3.378342,6.627678,-1.991411,0.439432,-1.445400,-1.005666,-1.509326,...,0.392094,6.117779,-4.025217,-1.892940,2.042058,4.636509,-2.782121,2.497840,1.721363,-3.491673
4,5.130456,11.372544,-0.707511,-1.410891,-5.441763,-7.412655,0.168574,1.198936,2.895670,-2.189699,...,-8.956708,-3.701063,9.969163,3.489908,4.024435,-8.425769,2.646266,-2.569123,3.154714,4.173816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,1.906505,10.356485,-5.350615,-4.707689,-5.816116,-5.515005,4.074656,9.743393,0.690833,-0.307510,...,-14.286254,-5.610931,7.831545,2.693676,3.701429,-14.963284,0.966225,-0.392336,6.641311,9.559189
254,-1.416878,4.316829,0.247929,4.398658,-0.325143,3.592517,-0.193848,-0.599689,1.999790,2.554787,...,-9.559757,-2.415578,7.254203,-0.362216,0.677062,-1.949787,-3.785800,-0.536386,-2.645318,11.154263
255,-2.882744,1.584436,1.659711,2.737474,1.201954,3.954856,-1.114793,-1.014974,3.478107,1.337362,...,-8.246337,-3.723928,7.963425,-0.508686,1.501254,-2.341476,-4.834466,-0.064883,0.564903,9.772692
256,-7.734784,-5.574593,-2.273346,0.334445,2.568140,8.763996,1.487155,2.768582,-0.477406,9.995866,...,-7.028116,1.817668,-0.531746,-2.376385,-3.008701,1.457805,-9.023492,3.969723,-2.321998,12.478557


In [79]:
predicted

,Bone-Osteosarc,Breast-AdenoCA,CNS-GBM,CNS-Medullo,CNS-PiloAstro,ColoRect-AdenoCA,Eso-AdenoCA,Head-SCC,Kidney-ChRCC,Kidney-RCC,...,Panc-AdenoCA,Panc-Endocrine,Prost-AdenoCA,Skin-Melanoma,Stomach-AdenoCA,Thy-AdenoCA,Uterus-AdenoCA,target,target_name,sample
0,3.926940,-4.404649,3.535940,-0.799068,-1.464853,-5.266735,2.989693,-2.446537,-5.237432,-6.868099,...,10.326580,2.196589,-3.409420,4.303656,-1.364747,-0.893581,-3.888031,[0],Bone-Osteosarc,['f83f1eb8-b2c8-c3d6-e040-11ac0c48261e']
1,10.247198,-0.113230,0.927531,2.949957,-0.457773,-4.395071,-0.388317,-5.213696,1.900337,-1.296216,...,5.348355,2.669073,0.935720,0.982028,4.515584,-0.692779,-0.150798,[0],Bone-Osteosarc,['f854ae3f-356d-f0ad-e040-11ac0c485dae']
2,11.536897,1.929139,0.892681,-2.113706,-5.948312,-6.383701,0.310385,-3.043523,-0.799770,1.794322,...,4.163595,5.533860,-5.426886,5.184233,3.791399,0.101267,0.783981,[0],Bone-Osteosarc,['f86ae246-2492-9785-e040-11ac0d4813b0']
3,-3.305912,-8.316203,-1.985560,2.321224,9.733523,0.827899,2.715782,1.767245,-0.495581,-1.660732,...,-1.666397,-1.811193,4.488020,-5.299366,2.108207,4.566350,-4.118330,[0],Bone-Osteosarc,['fc93e38d-6bd5-9912-e040-11ac0c4838cd']
4,5.842009,12.181273,0.007172,0.364507,-8.186638,-6.116848,-2.806248,2.948155,3.034819,0.391848,...,1.963405,4.051501,-7.419844,4.016503,-4.403675,1.931517,6.677391,[1],Breast-AdenoCA,['110b5f05-ded7-45ec-b0f6-a941c92d77dd']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,2.995948,11.558259,-2.636612,-2.917011,-6.300324,-2.755496,2.048807,9.772296,3.288156,1.058703,...,0.706878,2.769692,-12.648802,2.005544,-2.972754,6.227984,10.520501,[23],Uterus-AdenoCA,['0c39c03a-ec84-42c6-be69-8e98bc7a9f5a']
254,1.903786,3.223448,2.894243,2.945192,-1.998395,1.935413,0.062288,-1.176673,2.055380,-0.480946,...,3.159697,-0.622564,-4.271716,-4.338048,-1.748085,-1.225453,10.030005,[23],Uterus-AdenoCA,['493e7008-551c-4c0e-b567-a0f31868629e']
255,-2.532626,4.616404,2.234683,6.043796,-0.849505,5.355201,-5.286476,-2.922962,5.279253,3.330764,...,-3.291591,1.285418,0.935282,-8.049973,-1.719327,-2.342576,10.974082,[23],Uterus-AdenoCA,['82d2ea5f-ebdc-4907-a287-fe837fd6b59c']
256,-0.620141,-4.180835,0.311225,1.936613,3.365710,11.838383,-0.142388,-0.413681,3.507673,9.730452,...,-5.781797,-4.780820,2.897219,-10.614366,8.443665,-4.040802,11.962983,[23],Uterus-AdenoCA,['accfc45b-eae0-4991-a488-e217cdb46655']


In [80]:
logits = predicted.loc[:, 'Bone-Osteosarc':'Uterus-AdenoCA']
logits = logits.idxmax(1)

In [81]:
count_accuracy(predicted, files.loc[:,'nm_class'])

0.8875968992248062

In [82]:
truth = ground_truth.idxmax(1)

In [83]:
truth

0        Panc-AdenoCA
1      Bone-Osteosarc
2      Bone-Osteosarc
3       CNS-PiloAstro
4      Breast-AdenoCA
            ...      
253          Lung-SCC
254    Uterus-AdenoCA
255    Uterus-AdenoCA
256    Uterus-AdenoCA
257    Uterus-AdenoCA
Length: 258, dtype: object

In [84]:
df = pd.DataFrame({'truth': truth.values.tolist(), 'pred': logits.values.tolist()})
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(df)

                truth              pred
0        Panc-AdenoCA      Panc-AdenoCA
1      Bone-Osteosarc    Bone-Osteosarc
2      Bone-Osteosarc    Bone-Osteosarc
3       CNS-PiloAstro     CNS-PiloAstro
4      Breast-AdenoCA    Breast-AdenoCA
5      Breast-AdenoCA    Breast-AdenoCA
6      Breast-AdenoCA    Breast-AdenoCA
7      Breast-AdenoCA    Breast-AdenoCA
8            Lung-SCC    Breast-AdenoCA
9      Breast-AdenoCA     Ovary-AdenoCA
10     Breast-AdenoCA    Breast-AdenoCA
11     Breast-AdenoCA    Breast-AdenoCA
12     Breast-AdenoCA    Breast-AdenoCA
13     Breast-AdenoCA    Breast-AdenoCA
14     Breast-AdenoCA    Breast-AdenoCA
15     Breast-AdenoCA    Breast-AdenoCA
16     Breast-AdenoCA    Breast-AdenoCA
17       Panc-AdenoCA     Prost-AdenoCA
18     Breast-AdenoCA    Breast-AdenoCA
19     Breast-AdenoCA    Breast-AdenoCA
20     Breast-AdenoCA    Breast-AdenoCA
21     Breast-AdenoCA    Breast-AdenoCA
22     Breast-AdenoCA    Breast-AdenoCA
23     Breast-AdenoCA    Breast-AdenoCA


In [19]:
count_accuracy(ground_truth, files.loc[:,'nm_class'])

0.8687258687258688

## Copy sample list

In [71]:
files = pd.read_csv("/csc/epitkane/projects/litegpt/ckptpuhti/fullpcawgfold10_11100_wpos_TripletPosition_bs5000_nl1_nh1_ne128_cl3/listsample.csv")

In [72]:
files = pd.DataFrame({'samples':files.loc[:,'0'].values.tolist()})

In [74]:
targets

,Unnamed: 0,0
0,0,0
1,1,0
2,2,0
3,3,0
4,4,1
...,...,...
253,253,23
254,254,23
255,255,23
256,256,23


In [75]:
info = pd.read_csv("/csc/epitkane/projects/multimodal/data/utils/classinfo_pcawg_.csv")

In [76]:
names = targets.merge(info, left_on='0', right_on='class_index', how='left')

In [77]:
names = names.loc[:,'class_name'].values.tolist()

In [78]:
files['nm_class'] =names

In [79]:
files

,samples,nm_class
0,f83fc777-5416-c3e9-e040-11ac0d482c8e.csv,Bone-Osteosarc
1,f856fa85-fdb8-c0b0-e040-11ac0d480b4e.csv,Bone-Osteosarc
2,f86b6ba2-8f4b-ee53-e040-11ac0d485904.csv,Bone-Osteosarc
3,fc9469d0-4d0e-bb59-e040-11ac0d481e59.csv,Bone-Osteosarc
4,13c312ec-0add-4758-ab8d-c193e2e08c6d.csv,Breast-AdenoCA
...,...,...
253,105a51c4-cc7e-4d0f-9cf8-e4d64a31d14d.csv,Uterus-AdenoCA
254,554b9439-7e11-43ec-8bc3-448c59100b9e.csv,Uterus-AdenoCA
255,8d4cb709-c95c-4bdc-844b-c0bfa2a3028e.csv,Uterus-AdenoCA
256,af96db5a-684f-41d1-a910-5a5193393d9c.csv,Uterus-AdenoCA


In [80]:
files['slices'] = [10]*len(files)
files['fold'] = [10]*len(files)

In [82]:
files.to_csv("/csc/epitkane/projects/multimodal/extfiles/pcawg_val_test.csv")

In [83]:
files

,samples,nm_class,slices,fold
0,f83fc777-5416-c3e9-e040-11ac0d482c8e.csv,Bone-Osteosarc,10,10
1,f856fa85-fdb8-c0b0-e040-11ac0d480b4e.csv,Bone-Osteosarc,10,10
2,f86b6ba2-8f4b-ee53-e040-11ac0d485904.csv,Bone-Osteosarc,10,10
3,fc9469d0-4d0e-bb59-e040-11ac0d481e59.csv,Bone-Osteosarc,10,10
4,13c312ec-0add-4758-ab8d-c193e2e08c6d.csv,Breast-AdenoCA,10,10
...,...,...,...,...
253,105a51c4-cc7e-4d0f-9cf8-e4d64a31d14d.csv,Uterus-AdenoCA,10,10
254,554b9439-7e11-43ec-8bc3-448c59100b9e.csv,Uterus-AdenoCA,10,10
255,8d4cb709-c95c-4bdc-844b-c0bfa2a3028e.csv,Uterus-AdenoCA,10,10
256,af96db5a-684f-41d1-a910-5a5193393d9c.csv,Uterus-AdenoCA,10,10


## Collect the dataset

In [4]:
base_files = ['/csc/epitkane/projects/litegpt/ckptpuhti/fullpcawgfold{}_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3/'.format(fold) for fold in range(1,11)]
print(base_files)

['/csc/epitkane/projects/litegpt/ckptpuhti/fullpcawgfold1_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3/', '/csc/epitkane/projects/litegpt/ckptpuhti/fullpcawgfold2_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3/', '/csc/epitkane/projects/litegpt/ckptpuhti/fullpcawgfold3_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3/', '/csc/epitkane/projects/litegpt/ckptpuhti/fullpcawgfold4_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3/', '/csc/epitkane/projects/litegpt/ckptpuhti/fullpcawgfold5_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3/', '/csc/epitkane/projects/litegpt/ckptpuhti/fullpcawgfold6_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3/', '/csc/epitkane/projects/litegpt/ckptpuhti/fullpcawgfold7_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3/', '/csc/epitkane/projects/litegpt/ckptpuhti/fullpcawgfold8_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3/', '/csc/epitkane/projects/litegpt/ckptpuhti/fullpcawgfold9_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne51

In [7]:
folds = []
for i, base in enumerate(base_files):
    targets = pd.read_csv(os.path.join(base, "target.csv"))
    files = pd.read_csv(os.path.join(base, "listsample.csv")) 
    files = pd.DataFrame({'samples':files.loc[:,'0'].values.tolist()})
    info = pd.read_csv("/csc/epitkane/projects/multimodal/data/utils/classinfo_pcawg_.csv")
    names = targets.merge(info, left_on='0', right_on='class_index', how='left')
    names = names.loc[:,'class_name'].values.tolist()
    files['nm_class'] =names
    files['slices'] = [i+1]*len(files)
    files['fold'] = [i+1]*len(files)
    folds.append(files)

In [9]:
val_set = pd.concat(folds, ignore_index=True)

In [10]:
val_set

,samples,nm_class,slices,fold
0,f221c897-6ad0-0df9-e040-11ac0c4813ef.csv,Bone-Osteosarc,1,1
1,f8407106-8a96-8f85-e040-11ac0d481656.csv,Bone-Osteosarc,1,1
2,f8572359-f29a-5945-e040-11ac0d4821cf.csv,Bone-Osteosarc,1,1
3,f86b7e84-7040-c751-e040-11ac0c485675.csv,Bone-Osteosarc,1,1
4,fc950c33-faa4-0241-e040-11ac0c486786.csv,Bone-Osteosarc,1,1
...,...,...,...,...
2582,105a51c4-cc7e-4d0f-9cf8-e4d64a31d14d.csv,Uterus-AdenoCA,10,10
2583,554b9439-7e11-43ec-8bc3-448c59100b9e.csv,Uterus-AdenoCA,10,10
2584,8d4cb709-c95c-4bdc-844b-c0bfa2a3028e.csv,Uterus-AdenoCA,10,10
2585,af96db5a-684f-41d1-a910-5a5193393d9c.csv,Uterus-AdenoCA,10,10


In [19]:
val_set.to_csv("/csc/epitkane/projects/multimodal/extfiles/fullpcawg_val_checkpoint_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3.csv")

In [12]:
train_set = pd.DataFrame()
for skip in range(10):
    for i in range(10):
        if i != skip:
            data_to_add = folds[i]
            data_to_add['fold'] = [skip+1]*len(data_to_add)
            train_set = pd.concat([train_set, data_to_add])
        

In [14]:
train_set

,samples,nm_class,slices,fold
0,f82d213f-9843-28eb-e040-11ac0d483e48.csv,Bone-Osteosarc,2,1
1,f841c488-6b8d-6645-e040-11ac0c481418.csv,Bone-Osteosarc,2,1
2,f8593ac0-9480-22a0-e040-11ac0d48697a.csv,Bone-Osteosarc,2,1
3,f86e2d80-911b-7a19-e040-11ac0d486900.csv,Bone-Osteosarc,2,1
4,fc998aec-57dd-eab2-e040-11ac0d48680a.csv,Bone-Osteosarc,2,1
...,...,...,...,...
253,0c39c03a-ec84-42c6-be69-8e98bc7a9f5a.csv,Uterus-AdenoCA,9,10
254,493e7008-551c-4c0e-b567-a0f31868629e.csv,Uterus-AdenoCA,9,10
255,82d2ea5f-ebdc-4907-a287-fe837fd6b59c.csv,Uterus-AdenoCA,9,10
256,accfc45b-eae0-4991-a488-e217cdb46655.csv,Uterus-AdenoCA,9,10


In [26]:
train_set = train_set.sort_values(['fold', 'nm_class'])
#train_set = train_set.sort_values(['fold'])

In [27]:
train_set

,samples,nm_class,slices,fold
0,f82d213f-9843-28eb-e040-11ac0d483e48.csv,Bone-Osteosarc,2,1
1,f854ae3f-356d-f0ad-e040-11ac0c485dae.csv,Bone-Osteosarc,9,1
0,f82d213f-caa7-fd59-e040-11ac0d483e46.csv,Bone-Osteosarc,6,1
0,f83f1eb8-b2c8-c3d6-e040-11ac0c48261e.csv,Bone-Osteosarc,9,1
3,f8809cee-8f81-28f8-e040-11ac0d4873bf.csv,Bone-Osteosarc,8,1
...,...,...,...,...
258,eae45fbe-9f7d-4f59-a0a9-1f0f0d69afc4.csv,Uterus-AdenoCA,2,10
254,19def402-fb8d-4b27-90dd-ce9135a3c0a2.csv,Uterus-AdenoCA,3,10
255,638e80c7-9a6e-4a32-a621-fc4168e72343.csv,Uterus-AdenoCA,3,10
256,accfc45b-eae0-4991-a488-e217cdb46655.csv,Uterus-AdenoCA,9,10


In [28]:
train_set.to_csv("/csc/epitkane/projects/multimodal/extfiles/fullpcawg_train_checkpoint_11100_wpos_TripletPosition_bs5000_nl1_nh2_ne512_cl3.csv")

## Check model

In [26]:
train_info = pd.read_csv("/csc/epitkane/projects/multimodal/models/MuAtMotifPos_orig_10_fold_fixed_zero_grad/fold_1/MuAtMotifPos_orig_10_fold_fold1.csv.gz", compression='gzip')

In [30]:
np.max(train_info.loc[:,'vall_acc'].values)

0.8494208494208494

In [31]:
best_logits = pd.read_csv("/csc/epitkane/projects/multimodal/models/MuAtMotifPos_orig_10_fold_fixed_zero_grad/fold_1/val_logits_fold1_best_vallogits.tsv.gz", compression='gzip', sep='\t')

In [35]:
def count_accuracy(df):
    correct = 0
    targets = df.loc[:,'target']
    logits = df.loc[:,'Bone-Osteosarc':'Uterus-AdenoCA']
    for i in range(0, len(df)):
        target = int(targets.iloc[i].strip('[]'))
        row_logits = logits.loc[i,:]
        #print(row_logits)
        pred = row_logits.values.argmax()
        #print(pred)
        #print(target)
        if target == pred:
            correct += 1
    return correct/len(df)

In [36]:
count_accuracy(best_logits)

0.752895752895753

In [38]:
train_info.loc[:,'vall_acc']

0      0.474903
1      0.583012
2      0.640927
3      0.664093
4      0.710425
         ...   
145    0.776062
146    0.783784
147    0.803089
148    0.776062
149    0.752896
Name: vall_acc, Length: 150, dtype: float64

## crete context data

In [7]:
paths = []
paths = list_files_in_dir('/csc/epitkane/projects/multimodal/data/train_new/DNABERT_motif1001', paths)
#paths = list_files_in_dir('/csc/epitkane/projects/multimodal/train/motif3', paths)
paths.sort()

In [8]:
arrays = [i for i in paths if i.split('/')[-1][-4:] == '.npz' ]

In [19]:
base = '/csc/epitkane/projects/multimodal/data/train_new/DNABERT_1k_context'
for array_path in tqdm(arrays):
    arr = np.load(array_path)
    A = arr['motif']
    A = A[:, :768]
    file = '1k_context_' + array_path.split('/')[-1].split('.')[0] + '.txt.gz'
    folder = os.path.join(base, array_path.split('/')[-3], array_path.split('/')[-2])
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    np.savetxt(os.path.join(folder, file), A)

  0%|          | 0/7651 [00:00<?, ?it/s]

KeyboardInterrupt: 